In [ ]:
import torch
import numpy as np
from hydra import compose, initialize
from omegaconf import OmegaConf
import copy
import tqdm
from open_anything_diffusion.metrics.trajectory import flow_metrics, normalize_trajectory
initialize(config_path="../../configs", version_base="1.3")
cfg = compose(config_name="eval_sim")

In [ ]:
# from open_anything_diffusion.models.modules.dit_models import DGDiT
# from open_anything_diffusion.models.flow_diffuser_dgdit import (
#     FlowTrajectoryDiffuserSimulationModule_DGDiT,
# )
# network = DGDiT(
#     in_channels=3,
#     depth=5,
#     hidden_size=128,
#     patch_size=1,
#     num_heads=4,
#     n_points=cfg.dataset.n_points,
# ).cuda()
# model = FlowTrajectoryDiffuserSimulationModule_DGDiT(
#     network, inference_cfg=cfg.inference, model_cfg=cfg.model
# ).cuda()
# model.eval()

# ckpt = torch.load('/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_dgdit/2024-03-10/10-57-58/checkpoints/epoch=364-step=63875-val_loss=0.00-weights-only.ckpt')
# network.load_state_dict(
#     {k.partition(".")[2]: v for k, v, in ckpt["state_dict"].items()}
# )

In [ ]:
# import torch
# from open_anything_diffusion.models.modules.dit_models import DiT
# from open_anything_diffusion.models.flow_diffuser_dit import (
#     FlowTrajectoryDiffuserSimulationModule_DiT,
# )
# network = DiT(
#     in_channels=3 + 3,
#     depth=5,
#     hidden_size=128,
#     num_heads=4,
#     learn_sigma=True,
# ).cuda()
# model = FlowTrajectoryDiffuserSimulationModule_DiT(
#     network, inference_cfg=cfg.inference, model_cfg=cfg.model
# ).cuda()
# model.eval()

# ckpt = torch.load("/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_dit/2024-03-30/07-12-41/checkpoints/epoch=359-step=199080-val_loss=0.00-weights-only.ckpt",)
# network.load_state_dict(
#     {k.partition(".")[2]: v for k, v, in ckpt["state_dict"].items()}
# )

In [ ]:
# import rpad.pyg.nets.pointnet2 as pnp
# from open_anything_diffusion.models.flow_trajectory_predictor import FlowSimulationInferenceModule
# network = pnp.PN2Dense(
#     in_channels=0,
#     out_channels=3 * cfg.inference.trajectory_len,
#     p=pnp.PN2DenseParams(),
# )
# ckpt = torch.load('/home/yishu/open_anything_diffusion/logs/train_trajectory_pn++/2024-03-18/12-13-31/checkpoints/epoch=78-step=3476-val_loss=0.00-weights-only.ckpt')
# # ckpt = torch.load('/home/yishu/open_anything_diffusion/logs/train_trajectory_pn++/2024-05-26/02-37-08/checkpoints/epoch=98-step=109395-val_loss=0.00-weights-only.ckpt')
# network.load_state_dict(
#     {k.partition(".")[2]: v for k, v, in ckpt["state_dict"].items()}
# )
# model = FlowSimulationInferenceModule(
#     network, mask_input_channel=False
# )
# model.eval()

In [ ]:
from open_anything_diffusion.models.modules.dit_models import PN2HisDiT
from open_anything_diffusion.models.modules.history_encoder import HistoryEncoder
from open_anything_diffusion.models.flow_diffuser_hispndit import (
    FlowTrajectoryDiffuserSimulationModule_HisPNDiT,
)

class InferenceConfig:
    def __init__(self):
        self.batch_size = 1
        self.trajectory_len = 1
        self.mask_input_channel = False

inference_config = InferenceConfig()

class ModelConfig:
    def __init__(self):
        self.num_train_timesteps = 100

model_config = ModelConfig()

network = {
    "DiT": PN2HisDiT(
        history_embed_dim=128,
        in_channels=3,
        depth=5,
        hidden_size=128,
        num_heads=4,
        # depth=8,
        # hidden_size=256,
        # num_heads=4,
        learn_sigma=True,
    ).cuda(),
    "History": HistoryEncoder(
        history_dim=128,
        history_len=1,
        batch_norm=True,
        transformer=False,
        repeat_dim=False,
    ).cuda(),
}
ckpt_file = "/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_hispndit/2024-05-25/02-00-54/checkpoints/epoch=399-step=331600-val_loss=0.00-weights-only.ckpt"
model = FlowTrajectoryDiffuserSimulationModule_HisPNDiT(
    network, inference_cfg=inference_config, model_cfg=model_config
).cuda()
model.load_from_ckpt(ckpt_file)
model.eval()

In [ ]:
door_ids = ["8877", "9065", "9410", "8867", "9388", "8893"]
obj_id = door_ids[3]
# obj_id = "12248"

In [ ]:
import os
from rpad.partnet_mobility_utils.data import PMObject
pm_dir = os.path.expanduser("~/datasets/partnet-mobility/raw")
# env = PMSuctionSim(obj_id, pm_dir, gui=gui)
raw_data = PMObject(os.path.join(pm_dir, obj_id))

In [ ]:
available_joints = raw_data.semantics.by_type(
    "hinge"
) + raw_data.semantics.by_type("slider")
print(available_joints)

In [ ]:
available_joints = [joint.name for joint in available_joints]

In [ ]:
from open_anything_diffusion.simulations.suction import GTFlowModel, PMSuctionSim
env = PMSuctionSim(obj_id, pm_dir, gui=False)
gt_model = GTFlowModel(raw_data, env)

env.reset()

In [ ]:
import pybullet as p

for joint in available_joints:
    info = p.getJointInfo(
        env.render_env.obj_id,
        env.render_env.link_name_to_index[joint],
        env.render_env.client_id,
    )
    init_angle, target_angle = info[8], info[9]
    env.set_joint_state(joint, init_angle)
    # print(init_angle, target_angle)

In [ ]:
target_link = available_joints[0]
info = p.getJointInfo(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    env.render_env.client_id,
)
init_angle, target_angle = info[8], info[9]

## Initial angle

In [ ]:
env.set_joint_state(target_link, init_angle + np.linspace(0, 1, 20)[-1] * (target_angle - init_angle))
# env.set_joint_state(target_link, target_angle)
pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

In [ ]:
link_ixs = pc_seg == env.render_env.link_name_to_index[target_link]
gt_flow = gt_model(pc_obs)
print(gt_flow.shape)

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world]),
    torch.as_tensor([np.zeros((1200, 3))]),
    "red",
)
fig = animation.animate()
fig.show()

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world[link_ixs]),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world[link_ixs]]),
    torch.as_tensor([gt_flow[link_ixs].numpy() * 2]),
    "red",
)
fig = animation.animate()
fig.show()

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
colors = np.random.rand(len(link_ixs), 3)  # Shape (1000, 3)
# Convert the random colors to hexadecimal format
colors_hex = ['rgb({}, {}, {})'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors]
animation.add_trace(
    torch.as_tensor(P_world[link_ixs]),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world[link_ixs]]),
    torch.as_tensor([np.random.randn(len(gt_flow[link_ixs]), 3)]),
    # torch.as_tensor([gt_flow[link_ixs].numpy() * 2]),
    # colors_hex,
    "red"
)
fig = animation.animate()
fig.show()

In [ ]:
# import plotly.graph_objects as go
# colors = np.random.rand(1200, 3)  # Shape (1000, 3)

# # Convert the random colors to hexadecimal format
# colors_hex = ['rgb({}, {}, {})'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors]
# fig = go.Figure(data=[go.Scatter3d(
#     x=P_world[:, 0],
#     y=P_world[:, 1],
#     z=P_world[:, 2],
#     mode='markers',
#     marker=dict(
#         size=5,
#         color=colors_hex,  # Set color to the array of random colors
#         opacity=0.8
#     )
# )])

# # Step 4: Set plot titles
# fig.update_layout(
#     title='Predicted Noise',
#     scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z'
#     )
# )

# # Step 5: Show plot
# fig.show()

import numpy as np
import matplotlib.pyplot as plt
colors = np.random.randn(50, 3)
plt.figure(figsize=(3, 6))
plt.imshow(colors, aspect='auto', cmap='gray')
plt.colorbar(label='Value')
plt.title('1200 * 3 Matrix Visualized as 120 * 30 Heatmap')
plt.xlabel('dim = 3 (x, y, z)')
plt.ylabel('dim = point count')
plt.show()

## Try different open angles

In [ ]:
print(init_angle, np.linspace(init_angle, target_angle, 10), target_angle)

In [ ]:
angle_sample_cnts = 100
trial_cnts = 20 # 1
xs = []
rmses = []
cosines = []

past_gt_flow = None
past_pcd = None

for angle_ratio, angle in zip(
        np.linspace(0, 100, angle_sample_cnts),
        np.linspace(init_angle, target_angle, angle_sample_cnts)
    ):
    # print(angle_ratio)
    env.set_joint_state(target_link, angle)
    # env.set_joint_state(target_link, target_angle)
    pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
    rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

    gt_flow = gt_model(pc_obs)
    # nonzero_gt_flowixs = torch.where(gt_flow.norm(dim=-1) != 0.0)
    nonzero_gt_flowixs = pc_seg == env.render_env.link_name_to_index[target_link]
    gt_flow_nz = gt_flow[nonzero_gt_flowixs]

    for i in tqdm.tqdm(range(trial_cnts)):
        with torch.no_grad():
            # pred_flow = model(copy.deepcopy(pc_obs))[:, 0, :]
            pred_flow = model(
                copy.deepcopy(pc_obs),
                history_pcd=past_pcd,
                history_flow=past_gt_flow,
            )[:, 0, :]

        pred_flow_nz = pred_flow[nonzero_gt_flowixs]

        xs.append(angle_ratio)
        # cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
        # ys.append(cos_dist)

        rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
        rmses.append(rmse)
        cosines.append(cos_dist)

    past_gt_flow = gt_flow.cpu().numpy()
    past_pcd = P_world


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.scatter(xs, cosines, c='gray', s=20, alpha=0.2, edgecolors='black')
ax.scatter(xs, cosines, c='orange', s=20, alpha=0.2, edgecolors='red')
plt.title(f'Door {obj_id}')
plt.xlabel('Open ratio (%)')
plt.ylabel('Cosine similarity')
# plt.ylabel('Mag error (%)')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# ax.scatter(xs, rmses, c='gray', s=20, edgecolors='black')
ax.scatter(xs, rmses, c='orange', s=20, alpha=0.2, edgecolors='red')
plt.title(f'Door {obj_id}')
plt.xlabel('Open ratio (%)')
plt.ylabel('RMSE')
# plt.ylabel('Mag error (%)')

In [ ]:
import copy
cosines_flowbot = copy.deepcopy(cosines)
rmses_flowbot = copy.deepcopy(rmses)

### Prediction flow plot

In [ ]:
angle_ratio = 0
angle = init_angle + angle_ratio * (target_angle - init_angle)

In [ ]:
env.set_joint_state(target_link, angle - (0.2 * (target_angle - init_angle)))
# env.set_joint_state(target_link, target_angle)
pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

gt_flow = gt_model(pc_obs)
history_pcd = P_world
history_flow = gt_flow

In [ ]:
env.set_joint_state(target_link, angle)
pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

In [ ]:
print(angle_ratio)
# env.set_joint_state(target_link, target_angle)

gt_flow = gt_model(pc_obs)
# nonzero_gt_flowixs = torch.where(gt_flow.norm(dim=-1) != 0.0)
nonzero_gt_flowixs = pc_seg == env.render_env.link_name_to_index[target_link]
gt_flow_nz = gt_flow[nonzero_gt_flowixs]

with torch.no_grad():
    pred_flow, intermediates = model(copy.deepcopy(pc_obs), return_intermediate=True)
    pred_flow = pred_flow[:, 0, :]
    # pred_flow = model(copy.deepcopy(pc_obs), history_pcd, history_flow.numpy())[:, 0, :]

pred_flow_nz = pred_flow[nonzero_gt_flowixs]

# xs.append(angle_ratio)
# # cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
# # ys.append(cos_dist)

rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
# rmses.append(rmse)
# cosines.append(cos_dist)
print(rmse, cos_dist, mag_error)


#### Diffusion process

In [ ]:
intermediates[0].squeeze().flatten(start_dim=1, end_dim=2).permute(1, 0).shape

In [ ]:
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
from open_anything_diffusion.metrics.trajectory import normalize_trajectory
animation = FlowNetAnimation()
for i in range(101):
    if i % 1 != 0:
        continue
    flow = intermediates[i].squeeze().flatten(start_dim=1, end_dim=2).permute(1, 0).unsqueeze(1)
    flow = normalize_trajectory(flow)
    print(flow.shape)
    
    # print(flow[mask])
    # segmented_flow = np.zeros_like(flow, dtype=np.float32)
    # segmented_flow[mask] = flow[mask]
    # print("seg", segmented_flow, "flow", flow)
    animation.add_trace(
        torch.as_tensor(P_world),
        # torch.as_tensor([pcd[mask]]),
        # torch.as_tensor([flow[mask]]),
        torch.as_tensor([P_world]),
        torch.as_tensor([flow[:, 0, :].cpu().numpy() * 3]),
        "red",
    )

fig = animation.animate()
fig.show()

#### Diffusion visual

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world[nonzero_gt_flowixs]]),
    torch.as_tensor([pred_flow_nz.cpu().numpy() * 2]),
    "red",
)
fig = animation.animate()
fig.show()

### Grasp point

In [ ]:
import copy
import tqdm
from open_anything_diffusion.metrics.trajectory import flow_metrics

angle_sample_cnts = 20
trial_cnts = 100
xs = []
ys = []

for angle_ratio, angle in zip(
        np.linspace(5, 100, angle_sample_cnts),
        np.linspace(init_angle, target_angle, angle_sample_cnts)
    ):
    print(angle_ratio)
    env.set_joint_state(target_link, angle)
    # env.set_joint_state(target_link, target_angle)
    pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
    rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

    gt_flow = gt_model(pc_obs)
    affordance = np.zeros_like(gt_flow)
    # nonzero_gt_flowixs = torch.where(gt_flow.norm(dim=-1) != 0.0)
    nonzero_gt_flowixs = pc_seg == env.render_env.link_name_to_index[target_link]
    gt_flow_nz = gt_flow[nonzero_gt_flowixs]

    for i in tqdm.tqdm(range(trial_cnts)):
        with torch.no_grad():
            pred_flow = model(copy.deepcopy(pc_obs))[:, 0, :]

        pred_flow_nz = pred_flow[nonzero_gt_flowixs]
        flow_norm = pred_flow.norm(p=2, dim=-1)
        max_point_id = np.argmax(flow_norm)
        affordance[max_point_id][0] += 1

        xs.append(angle_ratio)
        # cos_dist = torch.cosine_similarity(pred_flow_nz, gt_flow_nz, dim=-1).mean()
        # ys.append(cos_dist)

        rmse, cos_dist, mag_error = flow_metrics(pred_flow_nz, gt_flow_nz)
        ys.append(mag_error)

    break

In [ ]:
affordance.shape

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world]),
    # torch.as_tensor([np.array(gt_flow)]),
    torch.as_tensor([affordance]),
    "red",
)
fig = animation.animate()
fig.show()